In [132]:
import cobra
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import refinegems

## Fix problem with Infeasible Error in get_secretion

In [66]:
#model = cobra.io.load_model('iJO1366')

testpath = "/Users/brune/Downloads/iAF692.xml"
model = refinegems.utility.io.load_model(testpath,'cobra')

In [67]:
with model:

    sf = model.summary().secretion_flux
    s = sf[sf['flux'] < 0.0].index.tolist()

    print(s)

['EX_ch4_e', 'EX_ch4s_e', 'EX_co2_e', 'EX_gcald_e', 'EX_gly_e', 'EX_h2o_e', 'EX_h_e', 'EX_unknown_rbfdeg_e']


In [68]:
model.optimize()

,fluxes,reduced_costs
ASPCT,0.009144,1.387779e-17
ASPK,0.034231,-1.387779e-17
ASPTA,-0.079818,4.751337e-18
ASPTRS,0.000000,4.163336e-17
ATGH,0.000018,-1.942890e-16
...,...,...
VALt2r,0.000000,-1.252600e-18
VOR,0.000000,3.320615e-18
XPPT,0.000000,-3.874205e-02
YUMPS,0.000000,-0.000000e+00


## Deal with external media

In [2]:
from refinegems.classes.medium import read_substances_from_file, read_external_medium

/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [3]:
filepath = '/Users/brune/Documents/20_Projects/01_sprg/media-test/test.tsv'
subs = read_substances_from_file(filepath)
subs

,name,formula,flux,source,BiGG,VMH
0,Water,H2O,10.0,water,h20,h20


In [5]:
filepath = '/Users/brune/Documents/20_Projects/01_sprg/media-test/test.tsv'
medium = read_external_medium('file', path=filepath)
medium

In [10]:
medium.substance_table

,name,formula,flux,source,BiGG,VMH
0,Water,H2O,10.0,water,h20,h20


### Add MNX to database

In [1]:
import sqlite3
from refinegems.utility.databases import PATH_TO_DB
import pandas as pd
from tqdm import tqdm
from pathlib import Path

# DISCLAIMER:
# Database information from MetaNetX
# distributed under https://creativecommons.org/licenses/by/4.0/
# Citation: MetaNetX/MNXref: unified namespace for metabolites and biochemical reactions in the context of metabolic models
#           Sébastien Moretti, Van Du T Tran, Florence Mehl, Mark Ibberson, Marco Pagni
#           Nucleic Acids Research (2021), 49(D1):D570-D574

mnx_db_namespace = {'reac_prop': ('https://www.metanetx.org/cgi-bin/mnxget/mnxref/reac_prop.tsv', 
                                  ['id','mnx_equation','reference','ec-code','is_balanced','is_transport']),
                    'reac_xref': ('https://www.metanetx.org/cgi-bin/mnxget/mnxref/reac_xref.tsv',
                                  ['source','id','description']),
                    'chem_prop': ('https://www.metanetx.org/cgi-bin/mnxget/mnxref/chem_prop.tsv',
                                  ['id','name','reference','formula','charge','mass','InChI','InChIKey','SMILES']),
                    'chem_xref': ('https://www.metanetx.org/cgi-bin/mnxget/mnxref/chem_xref.tsv',
                                  ['source','id','description'])
                    }

# @TODO time warning + progress bar (waiting w/o info is tedious)
def update_mnx_namespaces(db:Path|str=PATH_TO_DB, chunksize=1):
    con = sqlite3.connect(db)
    for name,values in mnx_db_namespace.items():
        link,colnames = values
        mnx_table = []
        for chunk in tqdm(pd.read_csv(link, sep='\t', comment='#', 
                                      names=colnames, 
                                      chunksize=chunksize*1024), 
                                      desc=f'Downloading {name}',
                                      unit='B'):# progress bar will not work -> no totel length info
            mnx_table.append(chunk) 

        match name:
            # Reaction property table
            case 'reac_prop':
                total_len = sum([len(_) for _ in mnx_table])
                with tqdm(total=total_len, unit='entries', 
                          desc='Add to DB') as pbar:
                    for i,chunk in enumerate(mnx_table):
                        if i == 0:
                            exists = 'replace'
                        else:
                            exists = 'append'
                        chunk.to_sql(
                            'mnx_'+name, db_connection, 
                            if_exists=exists, index=False, 
                            dtype={'id':'TEXT PRIMARY KEY'}
                            )
                        pbar.update(len(chunk)) 
        
            # Reaction cross-reference table
            case 'reac_xref':
                cursor = db_connection.cursor()
                cursor.execute('DROP TABLE IF EXISTS mnx_reac_xref')
                empty_table = """ CREATE TABLE mnx_reac_xref (
                                  source TEXT,
                                  id TEXT,
                                  description TEXT,
                                  CONSTRAINT PK_mnx_reac_xref PRIMARY KEY (source,id)
                                  FOREIGN KEY(id) REFERENCES mnx_reac_prop(id)
                              );
                              """
                cursor.execute(empty_table)
                total_len = sum([len(_) for _ in mnx_table])
                with tqdm(total=total_len, unit='entries',
                          desc='Add to DB') as pbar:
                    for i,chunk in enumerate(mnx_table):
                        chunk.to_sql(
                            'mnx_'+name, db_connection, 
                            if_exists='append', index=False
                            )
                        pbar.update(len(chunk))
                
            # Metabolite properties table
            case 'chem_prop':
                total_len = sum([len(_) for _ in mnx_table])
                with tqdm(total=total_len, unit='entries',
                          desc='Add to DB') as pbar:
                    for i,chunk in enumerate(mnx_table):
                        if i == 0:
                            exists = 'replace'
                        else:
                            exists = 'append'
                        chunk.to_sql(
                            'mnx_'+name, db_connection, 
                            if_exists=exists, index=False, 
                            dtype={'id':'TEXT PRIMARY KEY'}
                            )
                        pbar.update(len(chunk))
            # @TODO : there seems to be a problem with the unique constraint and case-sensitivity
            case 'chem_xref':
                total_len = sum([len(_) for _ in mnx_table])
                cursor = db_connection.cursor()
                cursor.execute('DROP TABLE IF EXISTS mnx_chem_xref')
                empty_table = """ CREATE TABLE mnx_chem_xref (
                                  source TEXT,
                                  id TEXT,
                                  description TEXT,
                                  CONSTRAINT PK_mnx_chem_xref PRIMARY KEY (source,id)
                                  FOREIGN KEY(id) REFERENCES mnx_chem_prop(id)
                              );
                              """
                cursor.execute(empty_table)
                with tqdm(total=total_len, unit='entries',
                          desc='Add to DB') as pbar:
                    for i,chunk in enumerate(mnx_table):
                        chunk.to_sql(
                            'mnx_'+name, db_connection, 
                            if_exists='append', index=False
                            )
                        pbar.update(len(chunk))
    con.close()
    

# @TEST
def reset_database(database:Path|str=PATH_TO_DB):
    """Remove tables for certain databases to allow pushing of the database
    to GitHub (reduce size).

    Args:
        - database (Path | str, optional): 
            Path to the database. Defaults to PATH_TO_DB, the in-build database.
    """
    # establish connection
    con = sqlite3.connect(database)
    cursor = con.cursor()
    # remove MetaNetX tables
    cursor.execute('DROP TABLE IF EXISTS mnx_chem_xref')
    cursor.execute('DROP TABLE IF EXISTS mnx_reac_xref')
    cursor.execute('DROP TABLE IF EXISTS mnx_chem_prop')
    cursor.execute('DROP TABLE IF EXISTS mnx_reac_prop')
    # close connection
    con.close()


/Users/brune/miniconda3/envs/sprg/lib/python3.10/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [2]:
update_mnx_namespaces()

# for pushing: maybe add functions to only allow download from MNX locally as a setup

Add to DB: 100%|██████████| 2996510/2996510 [00:28<00:00, 105285.64entries/s]
